In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML

sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh = 8
refRatio = 2
c = 1.
CFL = 0.5
nt = 32
RK = 4
deriv = 'CD'
order = 1

In [3]:
if (RK == 1):
    TimeIntegratorFunc = ST.ForwardEuler
else:
    if (RK == 2):
        TimeIntegratorFunc = ST.MidpointMeth
    else:
        TimeIntegratorFunc = ST.RK4

if (deriv == 'U'):
    DiffFunc = TT.Upwind #ST.Upwind
else:
    DiffFunc = TT.CenterDiff #ST.CenterDiff

In [4]:
omega = BT.Grid(nh)
finehalf = list(np.arange(int(nh / refRatio)) + int(nh / refRatio))
omega.AddPatch(refRatio, finehalf)
degFreed = omega.degFreed
nh_min = omega.nh_min
nh_max = omega.nh_max
t, nt = ST.CalcTime(omega, CFL, c, nt = nt)
derivMat = OT.SpaceDeriv(omega, order, 'deriv')
spatOp = -c * derivMat
strings = omega.strings
timePropOp = LA2.expm(t * spatOp)
omegaF = BT.Grid(nh_max)
wavesF = WT.MakeWaves(omegaF)

spacing:
[0.125  0.125  0.125  0.125  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
[ 0.      0.      0.     -0.0625  0.      0.      0.      0.      0.      0.      0.      0.0625]


0
[ 0.   1.   0.   0.   0.   0.   0.   0.   0.   0.  -0.5 -0.5]
[ 0.0625  0.      0.      0.     -0.0625  0.      0.      0.      0.      0.      0.      0.    ]

[[ 0.          1.          0.          0.          0.          0.          0.          0.          0.          0.         -0.5        -0.5       ]
 [-1.          0.          1.          0.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.         -1.          0.          1.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.         -1.          0.          0.5         0.5         0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.         -0.66666667 -0.3

In [5]:
print('derivMat =\n', derivMat, '\n')
print('spatOp =\n', spatOp, '\n')
print('timePropOp =\n', timePropOp, '\n')

derivMat =
 [[ 0.          4.          0.          0.          0.          0.          0.          0.          0.          0.         -2.         -2.        ]
 [-4.          0.          4.          0.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.         -4.          0.          4.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.         -4.          0.          2.          2.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.         -5.33333333 -2.66666667  8.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.         -8.          0.          8.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -8.          0.          8.          0.     

In [6]:
def SpacePoly(omega, DiffFunc, deg, coefs = []):
    errorLoc = 'ERROR:\nTestTools:\nSpacePoly:\n'
    if (coefs == []):
        coefs = np.ones(deg + 1, float)
    else:
        errorMess = BT.CheckSize(deg, coefs, nName = 'deg', matricaName = 'coefs')
        if (errorMess != ''):
            sys.exit(errorLoc + errorMess)
    nh_max = omega.nh_max
    waves = WT.MakeWaves(omega)
    x = omega.xCell
    nullspace = OT.FindNullspace(omega, waves)
    waveform = 0
    waveDeriv = 0
    polyString = 'u(x)'
    print(polyString + ' = ')
    #derivString = '$\frac{\textup{d}}{}'
    for d in range(deg + 1):
        waveform = waveform + (coefs[d] * (x ** d))
        if (d > 0):
            waveDeriv = waveDeriv + (coefs[d] * (x ** (d - 1)))
        
        if (d == 0):
            monoString = ' = ' + str(coefs[d])
            print(monoString)
        else:
            if (d == 1):
                monoString = ' ' + str(coefs[d]) + 'x'
            else:
                monoString = ' ' + str(coefs[d]) + 'x^' + str(d)
        if (d != deg):
            monoString = monoString + ' +'
        print(polyString)
        polyString = polyString + monoString
        print(polyString)
    waveDerivTest = DiffFunc(omega, 0, waveform, -1, 1)
    print(x)
    print(polyString)
    print(waveform)
    print(waveDeriv)
    print(waveDerivTest)
    print('')
    return

In [7]:
SpacePoly(omega, DiffFunc, 1)

u(x) = 
 = 1.0
u(x)
u(x) = 1.0 +
u(x) = 1.0 +
u(x) = 1.0 + 1.0x
spacing:
[0.125  0.125  0.125  0.125  0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
[ 0.      0.      0.     -0.0625  0.      0.      0.      0.      0.      0.      0.      0.0625]


0
[ 0.   1.   0.   0.   0.   0.   0.   0.   0.   0.  -0.5 -0.5]
[ 0.0625  0.      0.      0.     -0.0625  0.      0.      0.      0.      0.      0.      0.    ]

[[ 0.          1.          0.          0.          0.          0.          0.          0.          0.          0.         -0.5        -0.5       ]
 [-1.          0.          1.          0.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.         -1.          0.          1.          0.          0.          0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.         -1.          0.          0.5         0.5         0.          0.          0.          0.          0.          0.